In [7]:
import aiohttp
import json
import asyncio
from IPython.display import display, Markdown, clear_output

ollama_url_chat = "http://localhost:11434/api/chat"
ollama_model_name = "llama3.2:latest"

chat_history = []

async def chat(prompt, stream=True):
    global chat_history
    
    chat_history.append({"role": "user", "content": prompt})
    payload = {"model": ollama_model_name, "messages": chat_history, "stream": stream}
    headers = {"Content-Type": "application/json"}

    response_text = ""
    buffer = ""

    async with aiohttp.ClientSession() as session:
        async with session.post(ollama_url_chat, headers=headers, data=json.dumps(payload)) as response:
            if stream:
                async for chunk in response.content.iter_any():
                    try:
                        data = json.loads(chunk.decode('utf-8'))
                        content = data.get("message", {}).get("content", "")
                        buffer += content

                        if len(buffer) > 10:
                            response_text += buffer
                            clear_output(wait=True)
                            display(Markdown(response_text))
                            buffer = ""
                            
                    except json.JSONDecodeError:
                        continue

                response_text += buffer
                clear_output(wait=True)
                display(Markdown(response_text))
            else:
                response_text = await response.text()
                display(Markdown(response_text))

    chat_history.append({"role": "assistant", "content": response_text})

    return response_text

In [9]:
_ = await chat("What is Facebook?")

Facebook is a social media platform that allows users to create profiles, connect with friends and family, share updates and photos, join groups and communities, and participate in online discussions. It was founded in 2004 by Mark Zuckerberg and has since become one of the most widely used social media platforms in the world.

In essence, Facebook is a virtual space where people can:

* Share their thoughts, feelings, and experiences with others
* Connect with friends, family, and acquaintances from all over the world
* Join communities based on shared interests or affiliations
* Access information and news about current events, trending topics, and popular culture

Facebook's core features include:

1. Profiles: Users can create a profile to share information about themselves.
2. News Feed: A curated feed of updates from friends and pages users follow.
3. Groups: Communities based on shared interests or affiliations.
4. Messages: Private messaging between friends and family members.
5. Events: Creation and RSVPs for events, such as birthdays, weddings, or concerts.

Over time, Facebook has evolved to include additional features, such as:

1. Facebook Marketplace: A platform for buying and selling items locally.
2. Facebook Live: Real-time video streaming for personal and professional use.
3. Facebook Stories: A feature that allows users to share behind-the-scenes content.

Today, Facebook is a major part of modern online life, with billions of users worldwide.

In [10]:
_ = await chat("What is it's vision?")

Facebook's vision is to "give people the power to build community and bring the world closer together." This vision is reflected in Mark Zuckerberg's statement: "Our mission is to give people the power to build community and make the world more open and connected."

More specifically, Facebook's long-term goals include:

1. **Building a global network**: Connect people from all over the world and create a platform where everyone can contribute.
2. **Empowering communities**: Create tools and platforms that help people connect with others who share similar interests, passions, or causes.
3. **Facilitating knowledge sharing**: Provide a space for people to share information, ideas, and expertise, and make it easily accessible to others.
4. **Promoting online safety and security**: Protect users' data and ensure that the platform is a safe and trustworthy place for everyone.
5. **Innovating new technologies**: Continuously innovate and experiment with new technologies and features to improve the user experience.

Facebook's vision is also reflected in its core values:

1. **Openness**: Be open to new ideas, perspectives, and connections.
2. **Community**: Foster a sense of belonging and connection among users.
3. **Innovation**: Continuously innovate and experiment with new technologies and features.
4. **Transparency**: Be transparent about data collection, usage, and policies.
5. **Accountability**: Take responsibility for the impact of the platform on society.

Overall, Facebook's vision is to create a global network that connects people, fosters community, and promotes knowledge sharing, innovation, and positive change.